In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import decomposition
from sklearn import model_selection
import category_encoders as ce
from sklearn import tree

In [ ]:
df = pd.read_csv('merged_data.csv')

In [ ]:

df['month'] = pd.DatetimeIndex(df.date).month
df['year'] = pd.DatetimeIndex(df.date).year
df.columns

Index(['Unnamed: 0', 'date', 'date_block_num', 'shop_id', 'item_id',
       'item_price', 'item_cnt_day', 'item_name_translated',
       'shop_name_translated', 'item_category_id',
       'item_category_name_translated', 'shop_location', 'month', 'year'],
      dtype='object')

In [ ]:
p_df = df[['date_block_num', 'shop_id', 'item_id',
       'item_price', 'item_cnt_day', 'item_category_id', 'shop_location', 'month', 'year']]

In [ ]:
def fun(x):
    if x == 'Online':
        return 1
    elif x == 'Digital':
        return 1
    else: return 0
p_df["online"] = p_df['shop_location'].apply(fun)
p_df.head()

<ipython-input-6-4bfc90768ebb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_df["online"] = p_df['shop_location'].apply(fun)


,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,shop_location,month,year,online
0,0,59,22154,999.0,1.0,37,Yaroslavl,2,2013,0
1,0,24,22154,999.0,1.0,37,Moscow,1,2013,0
2,33,24,22154,299.0,1.0,37,Moscow,10,2015,0
3,0,27,22154,999.0,1.0,37,Moscow,1,2013,0
4,30,27,22154,299.0,1.0,37,Moscow,12,2015,0


In [ ]:
#p_df_gr     = p_df.groupby(by=['shop_id', 'item_id','date_block_num'])['item_price', 'item_category_id', 'month', 'year']
p_df_gr_sum = p_df.groupby(by=['shop_id', 'item_id','date_block_num'])['item_cnt_day'].agg(['sum']).reset_index()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 10 columns):
 #   Column                Dtype  
---  ------                -----  
 0   date_block_num        int64  
 1   shop_id               int64  
 2   item_id               int64  
 3   item_price            float64
 4   item_cnt_day          float64
 5   shop_name_translated  object 
 6   item_category_id      int64  
 7   shop_location         object 
 8   month                 int64  
 9   year                  int64  
dtypes: float64(2), int64(6), object(2)
memory usage: 224.0+ MB


KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
p_df = p_df.drop('item_cnt_day', axis=1)

In [ ]:
final_df = p_df_gr_sum.merge(p_df, how='left')#, on=['shop_id', 'item_id'])
final_df = final_df.drop_duplicates()

In [ ]:
final_df.columns

Index(['shop_id', 'item_id', 'date_block_num', 'sum', 'item_price',
       'item_category_id', 'shop_location', 'month', 'year', 'online'],
      dtype='object')

In [ ]:
x = final_df[['shop_id','item_id', 'date_block_num', 'item_price','item_category_id', 'month', 'year', 'online']].values
y = final_df[['sum']].values

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x,y)

In [ ]:
model = tree.ExtraTreeRegressor()#BaseDecisionTree()
model.fit(x_train,y_train)

ExtraTreeRegressor()

In [ ]:
print("model accuracy ", model.score(x_test,y_test)*100, "%")

model accuracy  92.41971646540954 %


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=298b82ff-04db-439b-a215-4b06da317c17' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>